In [102]:
import my_funcs as mf

In [28]:
import pandas as pd
import verde as vd
import numpy as np
import geopandas as gpd
import rioxarray as rio
import pyproj


import matplotlib.pyplot as plt

gdb = '/home/ggrl/geodatabase/'

%matplotlib widget

In [4]:
# Lendo cama vertorial gerada no QGIS com poligonos de 6º x 4º [Folhas em escala de 1:1.000.000]
malha_cartog_1kk = gpd.read_file(gdb+'cartografia/malha_cartog_1kk_wgs84.shp')
malha_cartog_1kk.head(5)

,id,left,top,right,bottom,geometry
0,4.0,-78.0,-4.0,-72.0,-8.0,"POLYGON ((-78.00000 -4.00000, -72.00000 -4.000..."
1,5.0,-78.0,-8.0,-72.0,-12.0,"POLYGON ((-78.00000 -8.00000, -72.00000 -8.000..."
2,13.0,-72.0,4.0,-66.0,0.0,"POLYGON ((-72.00000 4.00000, -66.00000 4.00000..."
3,14.0,-72.0,0.0,-66.0,-4.0,"POLYGON ((-72.00000 0.00000, -66.00000 0.00000..."
4,15.0,-72.0,-4.0,-66.0,-8.0,"POLYGON ((-72.00000 -4.00000, -66.00000 -4.000..."


In [5]:
# Criando coluna Region [W,E,S,N]
bounds = malha_cartog_1kk.bounds 

malha_cartog_1kk['region'] = [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                                                       bounds['miny'],bounds['maxy'])]

In [6]:
# Indexando as folhas 1kk com função 'nomeador_grid()' descrita em './my_funcs.py' importado como 'mf'
df = pd.DataFrame(malha_cartog_1kk)
lista_grid = []
for index, row in df.iterrows():
    row['id_folha'] = (mf.nomeador_grid(row.region[0],row.region[1],   
                                        row.region[3],row.region[2]))
    lista_grid.append(row.id_folha)

malha_cartog_1kk['id_folha'] = lista_grid

In [7]:
malha_cartog_1kk.head(5)

,id,left,top,right,bottom,geometry,region,id_folha
0,4.0,-78.0,-4.0,-72.0,-8.0,"POLYGON ((-78.00000 -4.00000, -72.00000 -4.000...","(-78.0, -72.0, -8.0, -4.0)",SB18
1,5.0,-78.0,-8.0,-72.0,-12.0,"POLYGON ((-78.00000 -8.00000, -72.00000 -8.000...","(-78.0, -72.0, -12.0, -8.0)",SC18
2,13.0,-72.0,4.0,-66.0,0.0,"POLYGON ((-72.00000 4.00000, -66.00000 4.00000...","(-72.0, -66.0, 0.0, 4.0)",NA19
3,14.0,-72.0,0.0,-66.0,-4.0,"POLYGON ((-72.00000 0.00000, -66.00000 0.00000...","(-72.0, -66.0, -4.0, 0.0)",SA19
4,15.0,-72.0,-4.0,-66.0,-8.0,"POLYGON ((-72.00000 -4.00000, -66.00000 -4.000...","(-72.0, -66.0, -8.0, -4.0)",SB19


In [8]:
malha_cartog_1kk.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)

In [9]:
malha_cartog_1kk.to_file(gdb+'database.gpkg',driver='GPKG',layer='malha_cartog_1kk_wgs84')

In [10]:
malha_cartog_1kk.head()

,geometry,id_folha
0,"POLYGON ((-78.00000 -4.00000, -72.00000 -4.000...",SB18
1,"POLYGON ((-78.00000 -8.00000, -72.00000 -8.000...",SC18
2,"POLYGON ((-72.00000 4.00000, -66.00000 4.00000...",NA19
3,"POLYGON ((-72.00000 0.00000, -66.00000 0.00000...",SA19
4,"POLYGON ((-72.00000 -4.00000, -66.00000 -4.000...",SB19


In [11]:
import my_funcs as f

In [12]:
# Definindo Regions (W,E,S,N)
def regions(gdf):
    bounds = gdf.bounds 
    gdf['region'] = [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                                                bounds['miny'],bounds['maxy'])]

# Definindo nomes a partir da articulação sistematica de folhas de cartas. Construindo uma lista e definindo como uma series.
def nomeador_malha(gdf):
    df = pd.DataFrame(gdf)
    lista_grid = []
    for index, row in df.iterrows():
        row['id_folha'] = (f.nomeador_grid(row.region[0],row.region[1],
                                            row.region[3],row.region[2],escala=5))
        lista_grid.append(row.id_folha)

    gdf['id_folha'] = lista_grid

In [13]:
malha_cartog_25k = gpd.read_file(gdb+'cartografia/malha_cartog_25k_wgs84.shp')   # Escala de 1:25.000
malha_cartog_50k = gpd.read_file(gdb+'cartografia/malha_cartog_50k_wgs84.shp')   # Escala de 1:50.000
malha_cartog_100k = gpd.read_file(gdb+'cartografia/malha_cartog_100k_wgs84.shp') # Escala de 1:100.000

regions(malha_cartog_25k)
regions(malha_cartog_50k)
regions(malha_cartog_100k)

nomeador_malha(malha_cartog_25k)
nomeador_malha(malha_cartog_50k)
nomeador_malha(malha_cartog_100k)

# Percebe-se que há redundância no código, o que leva ao excesso de informação ser salva/lida e processada.

In [14]:
malha_cartog_50k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)
malha_cartog_25k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)
malha_cartog_100k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)

In [16]:
malha_cartog_50k.to_file(gdb+'database.gpkg',driver='GPKG',layer='malha_cartog_50k_wgs84')
malha_cartog_25k.to_file(gdb+'database.gpkg',driver='GPKG',layer='malha_cartog_25k_wgs84')
malha_cartog_100k.to_file(gdb+'database.gpkg',driver='GPKG',layer='malha_cartog_100k_wgs84')

In [15]:
malha_cartog_25k.head(3)

,geometry,id_folha
0,"POLYGON ((-72.87500 -8.37500, -72.75000 -8.375...",SC18_X_B_II_3_SE
1,"POLYGON ((-72.87500 -8.25000, -72.75000 -8.250...",SC18_X_B_II_3_NE
2,"POLYGON ((-72.75000 -8.37500, -72.62500 -8.375...",SC18_X_B_II_4_SW


In [17]:
g_1105 = pd.read_csv(gdb+'geof/g1105')

In [18]:
g_1105.head(5)

,KPERC,eU,eTH,X,Y,MDT,CTCOR,LATITUDE,LONGITUDE
0,0.060,0.597,0.587,378133.94,7345653.93,0.00,3.305,-23.996399,-46.198067
1,0.139,0.332,0.824,378061.80,7345654.98,0.00,5.257,-23.996384,-46.198776
2,1.490,0.399,6.987,377989.96,7345656.47,0.14,8.125,-23.996365,-46.199482
3,1.975,1.459,6.574,377917.82,7345657.63,2.46,11.506,-23.996349,-46.200191
4,2.497,1.755,8.473,377845.99,7345659.12,12.47,14.051,-23.996330,-46.200897


In [19]:
# Filtro a partir do ID da folha cartográfica
SF23_V_D_25k = malha_cartog_25k[malha_cartog_25k['id_folha'].str.contains("SF23_V_D")]
SF23_V_D_25k

,geometry,id_folha
10117,"POLYGON ((-46.50000 -21.50000, -46.37500 -21.5...",SF23_V_D_IV_1_NW
10120,"POLYGON ((-46.50000 -21.00000, -46.37500 -21.0...",SF23_V_D_I_1_NW
10123,"POLYGON ((-46.25000 -21.25000, -46.12500 -21.2...",SF23_V_D_I_4_NW
10124,"POLYGON ((-46.25000 -21.37500, -46.12500 -21.3...",SF23_V_D_I_4_SW
10125,"POLYGON ((-46.12500 -21.37500, -46.00000 -21.3...",SF23_V_D_I_4_SE
...,...,...
37148,"POLYGON ((-46.25000 -21.87500, -46.12500 -21.8...",SF23_V_D_IV_4_SW
37151,"POLYGON ((-46.37500 -21.87500, -46.25000 -21.8...",SF23_V_D_IV_3_SE
37631,"POLYGON ((-45.12500 -21.00000, -45.00000 -21.0...",SF23_V_D_III_2_NE
37632,"POLYGON ((-45.12500 -21.12500, -45.00000 -21.1...",SF23_V_D_III_2_SE


In [83]:
g_1105 = pd.read_csv(gdb+'geof/g1105')
geof_1039 = pd.read_csv(gdb+'geof/geof_1039')
g_1089 = pd.read_csv(gdb+'geof/g1089')

In [20]:
g_3022 = pd.read_csv(gdb+'geof/g3022.csv')

In [21]:
regions(SF23_V_D_25k)

/home/ggrl/Softwares/anaconda3/envs/mappingBot/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [22]:
df = SF23_V_D_25k

In [23]:
df

,geometry,id_folha,region
10117,"POLYGON ((-46.50000 -21.50000, -46.37500 -21.5...",SF23_V_D_IV_1_NW,"(-46.5, -46.375, -21.625, -21.5)"
10120,"POLYGON ((-46.50000 -21.00000, -46.37500 -21.0...",SF23_V_D_I_1_NW,"(-46.5, -46.375, -21.125, -21.0)"
10123,"POLYGON ((-46.25000 -21.25000, -46.12500 -21.2...",SF23_V_D_I_4_NW,"(-46.25, -46.125, -21.375, -21.25)"
10124,"POLYGON ((-46.25000 -21.37500, -46.12500 -21.3...",SF23_V_D_I_4_SW,"(-46.25, -46.125, -21.5, -21.375)"
10125,"POLYGON ((-46.12500 -21.37500, -46.00000 -21.3...",SF23_V_D_I_4_SE,"(-46.125, -46.0, -21.5, -21.375)"
...,...,...,...
37148,"POLYGON ((-46.25000 -21.87500, -46.12500 -21.8...",SF23_V_D_IV_4_SW,"(-46.25, -46.125, -22.0, -21.875)"
37151,"POLYGON ((-46.37500 -21.87500, -46.25000 -21.8...",SF23_V_D_IV_3_SE,"(-46.375, -46.25, -22.0, -21.875)"
37631,"POLYGON ((-45.12500 -21.00000, -45.00000 -21.0...",SF23_V_D_III_2_NE,"(-45.125, -45.0, -21.125, -21.0)"
37632,"POLYGON ((-45.12500 -21.12500, -45.00000 -21.1...",SF23_V_D_III_2_SE,"(-45.125, -45.0, -21.25, -21.125)"


In [30]:
chain_list = ['CTCOR','KPERC','eU','eTh','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTh':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTh':(), 'MDT':()}

# Iterando entre as regions da malha cartográfica

for index, row in df.iterrows():
    print(row.id_folha+' start')
    data = g_3022[vd.inside((g_3022.LONGITUDE, g_3022.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    if data.empty == True:
        print('none')
    else:
        # Iterando entre os canais de interpolação
        for i in chain_list:
            chain = vd.Chain([
                ('trend', vd.Trend(degree=1)),
                ('reduce', vd.BlockReduce(np.median, spacing=499)),
                ('spline', vd.Spline())
            ])
            
            chain.fit(coordinates, data[i])
            
            grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
            grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)
            tif_ = grids[i].rename(easting = 'x',northing='y')
            tif_.rio.to_raster('/home/ggrl/Desktop/grids/3022/'+i+'_'+row.id_folha+'.tif')


SF23_V_D_IV_1_NW start
SF23_V_D_I_1_NW start
none
SF23_V_D_I_4_NW start
SF23_V_D_I_4_SW start
SF23_V_D_I_4_SE start
SF23_V_D_I_4_NE start
SF23_V_D_I_1_SE start
SF23_V_D_I_3_NE start
SF23_V_D_I_3_SE start
SF23_V_D_I_3_NW start
SF23_V_D_I_1_SW start
SF23_V_D_II_1_SW start
SF23_V_D_I_2_SW start
SF23_V_D_I_2_NW start
none
SF23_V_D_I_2_SE start
SF23_V_D_I_1_NE start
none
SF23_V_D_II_1_NW start
none
SF23_V_D_I_2_NE start
none
SF23_V_D_V_4_SW start
SF23_V_D_V_3_SE start
SF23_V_D_V_3_SW start
SF23_V_D_V_4_NW start
SF23_V_D_V_3_NE start
SF23_V_D_VI_4_SE start
SF23_V_D_VI_4_SW start
SF23_V_D_VI_3_SE start
SF23_V_D_VI_4_NE start
SF23_V_D_II_1_NE start
none
SF23_V_D_II_4_SW start
SF23_V_D_II_2_NE start
none
SF23_V_D_II_2_NW start
none
SF23_V_D_VI_3_SW start
SF23_V_D_V_4_SE start
SF23_V_D_VI_3_NE start
SF23_V_D_VI_3_NW start
SF23_V_D_III_4_NW start
SF23_V_D_III_3_NE start
SF23_V_D_III_2_SW start
SF23_V_D_III_1_SE start
SF23_V_D_III_3_NW start
SF23_V_D_III_1_SW start
SF23_V_D_II_4_NE start
SF23_V_D_

In [31]:
SF23_25k = malha_cartog_25k[malha_cartog_25k['id_folha'].str.contains("SF23")]

In [34]:
regions(SF23_25k)
df = SF23_25k

In [37]:
chain_list = ['CTCOR','KPERC','eU','eTH','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTH':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTH':(), 'MDT':()}

# Iterando entre as regions da malha cartográfica

for index, row in df.iterrows():
    print(row.id_folha+' start')
    data = g_1105[vd.inside((g_1105.LONGITUDE, g_1105.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    if data.empty == True:
        print('none')
    if len(data) < 1000:
        print('less then 1000 points')
    else:
        # Iterando entre os canais de interpolação
        for i in chain_list:
            chain = vd.Chain([
                ('trend', vd.Trend(degree=1)),
                ('reduce', vd.BlockReduce(np.median, spacing=499)),
                ('spline', vd.Spline())
            ])
            
            chain.fit(coordinates, data[i])
            
            grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
            grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)
            tif_ = grids[i].rename(easting = 'x',northing='y')
            tif_.rio.to_raster('/home/ggrl/Desktop/grids/1105/100m_25k/'+i+'_'+row.id_folha+'.tif')


SF23_Y_A_I_1_NW start
none
less then 1000 points
SF23_Y_C_I_1_SW start
none
less then 1000 points
SF23_Y_C_I_1_NW start
none
less then 1000 points
SF23_Y_C_I_3_SW start
none
less then 1000 points
SF23_V_C_IV_3_SW start
none
less then 1000 points
SF23_V_C_IV_1_NW start
none
less then 1000 points
SF23_Y_C_I_3_SE start
none
less then 1000 points
SF23_Y_C_I_1_NE start
none
less then 1000 points
SF23_Y_A_IV_3_SE start
none
less then 1000 points
SF23_Y_C_IV_1_SE start
none
less then 1000 points
SF23_Y_C_IV_1_SW start
none
less then 1000 points
SF23_Y_A_IV_1_SE start
none
less then 1000 points
SF23_Y_A_IV_3_NE start
none
less then 1000 points
SF23_Y_C_I_3_NE start
none
less then 1000 points
SF23_Y_A_I_4_SW start
none
less then 1000 points
SF23_Y_A_I_1_NE start
none
less then 1000 points
SF23_Y_A_I_2_SW start
none
less then 1000 points
SF23_Y_A_I_2_NW start
none
less then 1000 points
SF23_Y_C_I_4_SE start
none
less then 1000 points
SF23_Y_C_I_4_SW start
none
less then 1000 points
SF23_Y_A_I_2_

In [38]:
gama_cols = 'KB DATA BARO UB THB COSMICO CTB UUP ALTURA KPERC eU eTH CTEXP UTHRAZAO X Y UKRAZAO MDT THKRAZAO LIVE_TIME CTCOR KCOR THCOR UCOR HORA GPSALT LATITUDE FIDUCIAL TEMP LONGITUDE'.split(" ")


gLine_1105 = pd.read_csv(gdb+'xyz/1105_XYZ/1105_GamaLine.XYZ',
                         names=gama_cols,
                         delim_whitespace=True,
                         skiprows=11,
                         usecols=["X","Y","LATITUDE","LONGITUDE",
                                  "KPERC","eU","eTH","CTCOR","MDT"])

In [39]:
df = malha_cartog_50k

In [42]:
regions(malha_cartog_50k)

In [43]:
df = malha_cartog_50k[malha_cartog_50k['id_folha'].str.contains("SF23")]

In [44]:
df

,geometry,id_folha,region
8399,"POLYGON ((-48.00000 -20.00000, -47.75000 -20.0...",SF23_V_A_I_1,"(-48.0, -47.75, -20.25, -20.0)"
8400,"POLYGON ((-48.00000 -20.25000, -47.75000 -20.2...",SF23_V_A_I_3,"(-48.0, -47.75, -20.5, -20.25)"
8401,"POLYGON ((-48.00000 -20.50000, -47.75000 -20.5...",SF23_V_A_IV_1,"(-48.0, -47.75, -20.75, -20.5)"
8402,"POLYGON ((-48.00000 -20.75000, -47.75000 -20.7...",SF23_V_A_IV_3,"(-48.0, -47.75, -21.0, -20.75)"
8403,"POLYGON ((-48.00000 -21.00000, -47.75000 -21.0...",SF23_V_C_I_1,"(-48.0, -47.75, -21.25, -21.0)"
...,...,...,...
10465,"POLYGON ((-42.25000 -22.00000, -42.00000 -22.0...",SF23_Z_B_III_2,"(-42.25, -42.0, -22.25, -22.0)"
10466,"POLYGON ((-42.25000 -22.25000, -42.00000 -22.2...",SF23_Z_B_III_4,"(-42.25, -42.0, -22.5, -22.25)"
10467,"POLYGON ((-42.25000 -22.50000, -42.00000 -22.5...",SF23_Z_B_VI_2,"(-42.25, -42.0, -22.75, -22.5)"
10468,"POLYGON ((-42.25000 -22.75000, -42.00000 -22.7...",SF23_Z_B_VI_4,"(-42.25, -42.0, -23.0, -22.75)"


In [48]:
chain_list = ['CTCOR','KPERC','eU','eTH','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTH':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTH':(), 'MDT':()}

{row.id_folha}

# Iterando entre as regions da malha cartográfica

for index, row in df.iterrows():
    print(row.id_folha+' start')
    data = gLine_1105[vd.inside((gLine_1105.LONGITUDE, gLine_1105.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    if data.empty == True:
        print('none')

    if len(data) < 1000:
        print('less then 1000 points')
    else:
        # Iterando entre os canais de interpolação
        for i in chain_list:
            chain = vd.Chain([
                ('trend', vd.Trend(degree=1)),
                ('reduce', vd.BlockReduce(np.median, spacing=499)),
                ('spline', vd.Spline())
            ])
            
            chain.fit(coordinates, data[i])
            
            grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
            grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)
            tif_ = grids[i].rename(easting = 'x',northing='y')
            tif_.rio.to_raster('/home/ggrl/Desktop/grids/1105/gline/'+i+'_'+row.id_folha+'.tif')


SF23_V_A_I_1 start
none
less then 1000 points
SF23_V_A_I_3 start
none
less then 1000 points
SF23_V_A_IV_1 start
none
less then 1000 points
SF23_V_A_IV_3 start
none
less then 1000 points
SF23_V_C_I_1 start
none
less then 1000 points
SF23_V_C_I_3 start
none
less then 1000 points
SF23_V_C_IV_1 start
none
less then 1000 points
SF23_V_C_IV_3 start
none
less then 1000 points
SF23_Y_A_I_1 start
none
less then 1000 points
SF23_Y_A_I_3 start
none
less then 1000 points
SF23_Y_A_IV_1 start
none
less then 1000 points
SF23_Y_A_IV_3 start
none
less then 1000 points
SF23_Y_C_I_1 start
none
less then 1000 points
SF23_Y_C_I_3 start
none
less then 1000 points
SF23_Y_C_IV_1 start
none
less then 1000 points
SF23_Y_C_IV_3 start
none
less then 1000 points
SF23_V_A_I_2 start
none
less then 1000 points
SF23_V_A_I_4 start
none
less then 1000 points
SF23_V_A_IV_2 start
none
less then 1000 points
SF23_V_A_IV_4 start
none
less then 1000 points
SF23_V_C_I_2 start
none
less then 1000 points
SF23_V_C_I_4 start
none


In [13]:
g_3022 = pd.read_csv(gdb+'geof/g3022.csv')

In [15]:
df = malha_cartog_50k[malha_cartog_50k['id_folha'].str.contains("SF23")]
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df

/home/ggrl/Softwares/anaconda3/envs/mappingBot/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,id_folha,geometry
0,SF23_V_A_I_1,"POLYGON ((-48.00000 -20.00000, -47.75000 -20.0..."
1,SF23_V_A_I_3,"POLYGON ((-48.00000 -20.25000, -47.75000 -20.2..."
2,SF23_V_A_IV_1,"POLYGON ((-48.00000 -20.50000, -47.75000 -20.5..."
3,SF23_V_A_IV_3,"POLYGON ((-48.00000 -20.75000, -47.75000 -20.7..."
4,SF23_V_C_I_1,"POLYGON ((-48.00000 -21.00000, -47.75000 -21.0..."
...,...,...
342,SF23_Z_B_III_2,"POLYGON ((-42.25000 -22.00000, -42.00000 -22.0..."
343,SF23_Z_B_III_4,"POLYGON ((-42.25000 -22.25000, -42.00000 -22.2..."
344,SF23_Z_B_VI_2,"POLYGON ((-42.25000 -22.50000, -42.00000 -22.5..."
345,SF23_Z_B_VI_4,"POLYGON ((-42.25000 -22.75000, -42.00000 -22.7..."


In [17]:
regions(df)

/home/ggrl/Softwares/anaconda3/envs/mappingBot/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [18]:
df

,id_folha,geometry,region
0,SF23_V_A_I_1,"POLYGON ((-48.00000 -20.00000, -47.75000 -20.0...","(-48.0, -47.75, -20.25, -20.0)"
1,SF23_V_A_I_3,"POLYGON ((-48.00000 -20.25000, -47.75000 -20.2...","(-48.0, -47.75, -20.5, -20.25)"
2,SF23_V_A_IV_1,"POLYGON ((-48.00000 -20.50000, -47.75000 -20.5...","(-48.0, -47.75, -20.75, -20.5)"
3,SF23_V_A_IV_3,"POLYGON ((-48.00000 -20.75000, -47.75000 -20.7...","(-48.0, -47.75, -21.0, -20.75)"
4,SF23_V_C_I_1,"POLYGON ((-48.00000 -21.00000, -47.75000 -21.0...","(-48.0, -47.75, -21.25, -21.0)"
...,...,...,...
342,SF23_Z_B_III_2,"POLYGON ((-42.25000 -22.00000, -42.00000 -22.0...","(-42.25, -42.0, -22.25, -22.0)"
343,SF23_Z_B_III_4,"POLYGON ((-42.25000 -22.25000, -42.00000 -22.2...","(-42.25, -42.0, -22.5, -22.25)"
344,SF23_Z_B_VI_2,"POLYGON ((-42.25000 -22.50000, -42.00000 -22.5...","(-42.25, -42.0, -22.75, -22.5)"
345,SF23_Z_B_VI_4,"POLYGON ((-42.25000 -22.75000, -42.00000 -22.7...","(-42.25, -42.0, -23.0, -22.75)"


In [20]:
g_3022

,CTCOR,eTh,eU,KPERC,LATITUDE,LONGITUDE,MDT,X,Y
0,644.27,6.9,-0.1,0.791,-21.236695,-47.054024,699.38,286856.47,7650282.76
1,577.39,6.5,-0.1,0.681,-21.237383,-47.054043,697.18,286855.49,7650206.55
2,520.33,5.9,-0.1,0.587,-21.238073,-47.054056,707.69,286855.13,7650130.13
3,494.39,5.3,0.1,0.566,-21.238763,-47.054064,721.80,286855.29,7650053.72
4,502.44,4.8,0.2,0.634,-21.239452,-47.054070,727.61,286855.66,7649977.42
...,...,...,...,...,...,...,...,...,...
676761,-21.96,0.7,-0.7,0.129,-21.069341,-44.001161,1067.13,603787.96,7669863.86
676762,10.95,1.0,-0.7,0.127,-21.069324,-44.000478,1058.53,603858.93,7669865.29
676763,19.16,1.1,-0.6,0.129,-21.069300,-43.999797,1065.43,603929.70,7669867.51
676764,1.20,0.8,-0.7,0.144,-21.069277,-43.999119,1087.47,604000.15,7669869.61


In [85]:
def select_area(escala,id):
    malha_cartog = mf.importar('malha_cartog_'+escala+'_wgs84')
    regions(malha_cartog)
    df = pd.DataFrame(malha_cartog[malha_cartog['id_folha'].str.contains(id)])
    return(df)

In [86]:
select_area('100K','SF23')

,id_folha,geometry,region
2193,SF23_V_A_I,"POLYGON ((-48.00000 -20.00000, -47.50000 -20.0...","(-48.0, -47.5, -20.5, -20.0)"
2194,SF23_V_A_IV,"POLYGON ((-48.00000 -20.50000, -47.50000 -20.5...","(-48.0, -47.5, -21.0, -20.5)"
2195,SF23_V_C_I,"POLYGON ((-48.00000 -21.00000, -47.50000 -21.0...","(-48.0, -47.5, -21.5, -21.0)"
2196,SF23_V_C_IV,"POLYGON ((-48.00000 -21.50000, -47.50000 -21.5...","(-48.0, -47.5, -22.0, -21.5)"
2197,SF23_Y_A_I,"POLYGON ((-48.00000 -22.00000, -47.50000 -22.0...","(-48.0, -47.5, -22.5, -22.0)"
...,...,...,...
2695,SF23_X_D_III,"POLYGON ((-42.50000 -21.00000, -42.00000 -21.0...","(-42.5, -42.0, -21.5, -21.0)"
2696,SF23_X_D_VI,"POLYGON ((-42.50000 -21.50000, -42.00000 -21.5...","(-42.5, -42.0, -22.0, -21.5)"
2697,SF23_Z_B_III,"POLYGON ((-42.50000 -22.00000, -42.00000 -22.0...","(-42.5, -42.0, -22.5, -22.0)"
2698,SF23_Z_B_VI,"POLYGON ((-42.50000 -22.50000, -42.00000 -22.5...","(-42.5, -42.0, -23.0, -22.5)"


In [33]:
df = pd.DataFrame(malha_cartog_25k[malha_cartog_25k['id_folha'].str.contains("SF23")])

In [34]:
df 

,right,geometry,region,id_folha
9819,-47.875,"POLYGON ((-48.00000 -22.00000, -47.87500 -22.0...","(-48.0, -47.875, -22.125, -22.0)",SF23_Y_A_I_1
9820,-47.875,"POLYGON ((-48.00000 -23.12500, -47.87500 -23.1...","(-48.0, -47.875, -23.25, -23.125)",SF23_Y_C_I_1
9821,-47.875,"POLYGON ((-48.00000 -23.00000, -47.87500 -23.0...","(-48.0, -47.875, -23.125, -23.0)",SF23_Y_C_I_1
9822,-47.875,"POLYGON ((-48.00000 -23.37500, -47.87500 -23.3...","(-48.0, -47.875, -23.5, -23.375)",SF23_Y_C_I_3
9823,-47.875,"POLYGON ((-48.00000 -21.87500, -47.87500 -21.8...","(-48.0, -47.875, -22.0, -21.875)",SF23_V_C_IV_3
...,...,...,...,...
46193,-46.875,"POLYGON ((-47.00000 -20.75000, -46.87500 -20.7...","(-47.0, -46.875, -20.875, -20.75)",SF23_V_A_VI_3
46194,-46.375,"POLYGON ((-46.50000 -20.75000, -46.37500 -20.7...","(-46.5, -46.375, -20.875, -20.75)",SF23_V_B_IV_3
46195,-46.625,"POLYGON ((-46.75000 -20.62500, -46.62500 -20.6...","(-46.75, -46.625, -20.75, -20.625)",SF23_V_A_VI_2
46196,-46.750,"POLYGON ((-46.87500 -20.50000, -46.75000 -20.5...","(-46.875, -46.75, -20.625, -20.5)",SF23_V_A_VI_1


In [35]:
chain_list = ['CTCOR','KPERC','eU','eTh','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTh':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTh':(), 'MDT':()}

# Iterando entre as regions da malha cartográfica
for index, row in df.iterrows():
    print(row.id_folha+' start')
    data = g_3022[vd.inside((g_3022.LONGITUDE, g_3022.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    if data.empty == True:  # if data dont have values pass to next step
        print('none')
    if len(data) < 2000:    # if less then 2000 points pass to next step
        print('few data')
    else:
        # Iterando entre os canais de interpolação
        for i in chain_list:
            print(i)
            chain = vd.Chain([
                ('trend', vd.Trend(degree=1)),
                ('reduce', vd.BlockReduce(np.median, spacing=499)),
                ('spline', vd.Spline())
            ])
            
            chain.fit(coordinates, data[i])
            
            grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
            grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)
            tif_ = grids[i].rename(easting = 'x',northing='y')
            tif_.rio.to_raster('/home/ggrl/Desktop/grids/3022/100m_25k/'+i+'_'+row.id_folha+'.tif')

SF23_Y_A_I_1 start
none
few data
SF23_Y_C_I_1 start
none
few data
SF23_Y_C_I_1 start
none
few data
SF23_Y_C_I_3 start
none
few data
SF23_V_C_IV_3 start
none
few data
SF23_V_C_IV_1 start
none
few data
SF23_Y_C_I_3 start
none
few data
SF23_Y_C_I_1 start
none
few data
SF23_Y_A_IV_3 start
none
few data
SF23_Y_C_IV_1 start
none
few data
SF23_Y_C_IV_1 start
none
few data
SF23_Y_A_IV_1 start
none
few data
SF23_Y_A_IV_3 start
none
few data
SF23_Y_C_I_3 start
none
few data
SF23_Y_A_I_4 start
none
few data
SF23_Y_A_I_1 start
none
few data
SF23_Y_A_I_2 start
none
few data
SF23_Y_A_I_2 start
none
few data
SF23_Y_C_I_4 start
none
few data
SF23_Y_C_I_4 start
none
few data
SF23_Y_A_I_2 start
none
few data
SF23_V_C_IV_4 start
none
few data
SF23_Y_A_IV_4 start
none
few data
SF23_Y_A_IV_4 start
none
few data
SF23_Y_C_IV_4 start
none
few data
SF23_Y_C_IV_4 start
none
few data
SF23_Y_A_I_2 start
none
few data
SF23_Y_C_IV_2 start
none
few data
SF23_Y_A_IV_4 start
none
few data
SF23_Y_A_I_4 start
none
few d

In [37]:
g_1105 = pd.read_csv(gdb+'geof/g1105')

In [42]:
chain_list = ['CTCOR','KPERC','eU','eTH','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTh':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eTh':(), 'MDT':()}

# Iterando entre as regions da malha cartográfica

for index, row in df.iterrows():
    print(row.id_folha+' start')
    data = g_1105[vd.inside((g_1105.LONGITUDE, g_1105.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    if len(data) < 200:    # if less then 200 points pass to next step
        print('few data')
    else:
        # Iterando entre os canais de interpolação
        for i in chain_list:
            print(i)
            chain = vd.Chain([
                ('trend', vd.Trend(degree=1)),
                ('reduce', vd.BlockReduce(np.median, spacing=499)),
                ('spline', vd.Spline())
            ])
            
            chain.fit(coordinates, data[i])
            
            grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
            grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)
            tif_ = grids[i].rename(easting = 'x',northing='y')
            tif_.rio.to_raster('/home/ggrl/Desktop/grids/1105/100m_25k/'+i+'_'+row.id_folha+'.tif')

SF23_Y_A_I_1 start
few data
SF23_Y_C_I_1 start
few data
SF23_Y_C_I_1 start
few data
SF23_Y_C_I_3 start
few data
SF23_V_C_IV_3 start
few data
SF23_V_C_IV_1 start
few data
SF23_Y_C_I_3 start
few data
SF23_Y_C_I_1 start
few data
SF23_Y_A_IV_3 start
few data
SF23_Y_C_IV_1 start
few data
SF23_Y_C_IV_1 start
few data
SF23_Y_A_IV_1 start
few data
SF23_Y_A_IV_3 start
few data
SF23_Y_C_I_3 start
few data
SF23_Y_A_I_4 start
few data
SF23_Y_A_I_1 start
few data
SF23_Y_A_I_2 start
few data
SF23_Y_A_I_2 start
few data
SF23_Y_C_I_4 start
few data
SF23_Y_C_I_4 start
few data
SF23_Y_A_I_2 start
few data
SF23_V_C_IV_4 start
few data
SF23_Y_A_IV_4 start
few data
SF23_Y_A_IV_4 start
few data
SF23_Y_C_IV_4 start
few data
SF23_Y_C_IV_4 start
few data
SF23_Y_A_I_2 start
few data
SF23_Y_C_IV_2 start
few data
SF23_Y_A_IV_4 start
few data
SF23_Y_A_I_4 start
few data
SF23_Y_C_IV_4 start
few data
SF23_Y_C_V_3 start
few data
SF23_Y_C_IV_4 start
few data
SF23_Y_C_V_3 start
few data
SF23_Y_C_V_1 start
few data
SF23

In [68]:
df

,geometry,id_folha,region
8399,"POLYGON ((-48.00000 -20.00000, -47.75000 -20.0...",SF23_V_A_I_1,"(-48.0, -47.75, -20.25, -20.0)"
8400,"POLYGON ((-48.00000 -20.25000, -47.75000 -20.2...",SF23_V_A_I_3,"(-48.0, -47.75, -20.5, -20.25)"
8401,"POLYGON ((-48.00000 -20.50000, -47.75000 -20.5...",SF23_V_A_IV_1,"(-48.0, -47.75, -20.75, -20.5)"
8402,"POLYGON ((-48.00000 -20.75000, -47.75000 -20.7...",SF23_V_A_IV_3,"(-48.0, -47.75, -21.0, -20.75)"
8403,"POLYGON ((-48.00000 -21.00000, -47.75000 -21.0...",SF23_V_C_I_1,"(-48.0, -47.75, -21.25, -21.0)"
...,...,...,...
10465,"POLYGON ((-42.25000 -22.00000, -42.00000 -22.0...",SF23_Z_B_III_2,"(-42.25, -42.0, -22.25, -22.0)"
10466,"POLYGON ((-42.25000 -22.25000, -42.00000 -22.2...",SF23_Z_B_III_4,"(-42.25, -42.0, -22.5, -22.25)"
10467,"POLYGON ((-42.25000 -22.50000, -42.00000 -22.5...",SF23_Z_B_VI_2,"(-42.25, -42.0, -22.75, -22.5)"
10468,"POLYGON ((-42.25000 -22.75000, -42.00000 -22.7...",SF23_Z_B_VI_4,"(-42.25, -42.0, -23.0, -22.75)"


In [165]:
def select_area(escala,id):
    malha_cartog = mf.importar('malha_cartog_'+escala+'_wgs84')
    regions(malha_cartog)
    df = pd.DataFrame(malha_cartog[malha_cartog['id_folha'].str.contains(id)])
    
    return(df)

In [204]:
df = select_area('50k','SF23_V_D_VI')

In [205]:
df

,id_folha,geometry,region
9354,SF23_V_D_VI_1,"POLYGON ((-45.50000 -21.50000, -45.25000 -21.5...","(-45.5, -45.25, -21.75, -21.5)"
9355,SF23_V_D_VI_3,"POLYGON ((-45.50000 -21.75000, -45.25000 -21.7...","(-45.5, -45.25, -22.0, -21.75)"
9445,SF23_V_D_VI_2,"POLYGON ((-45.25000 -21.50000, -45.00000 -21.5...","(-45.25, -45.0, -21.75, -21.5)"
9446,SF23_V_D_VI_4,"POLYGON ((-45.25000 -21.75000, -45.00000 -21.7...","(-45.25, -45.0, -22.0, -21.75)"


In [193]:
# Iterando entre as regions da malha cartográfica
def interpolar(df,geof,degree=1,spacing=499,psize=100,n_splits=15,validate=False):
    chain_list = ['CTCOR','eU','eTh','MDT','KPERC']      # Lista de canais a serem interpolados
                                                      
    grids = {chain_list[0]:(),chain_list[1]:(),chain_list[2]:(),chain_list[3]:(), chain_list[4]:()}    # Dicionário para salvar os dados interpolados (grids)
    scores = {chain_list[0]:(),chain_list[1]:(),chain_list[2]:(), chain_list[3]:(), chain_list[4]:()}   # Dicionário para salvar os dados da validação cruzada
    
    
       
    for index, row in df.iterrows():
        print(row.id_folha+' start')
        data = geof[vd.inside((geof.LONGITUDE, geof.LATITUDE), region = row.region)]
        coordinates = (data.X.values, data.Y.values)
        if data.empty == True:  # if data dont have values pass to next step
            print('none')
        if len(data) < 2000:    # if less then 2000 points pass to next step
            print('few data')
        else:
            # Iterando entre os canais de interpolação
            for i in chain_list:
                print(i)
                chain = vd.Chain([
                    ('trend', vd.Trend(degree=degree)),
                    ('reduce', vd.BlockReduce(np.median, spacing=spacing)),
                    ('spline', vd.Spline())
                ])
                
                chain.fit(coordinates, data[i])
                
                if validate == True:
                    cv     = vd.BlockKFold(spacing=spacing,
                                n_splits=n_splits,
                                shuffle=True)

                    scores[i] = vd.cross_val_score(chain,
                                            coordinates,
                                            data[i],
                                            cv=cv)
                    

                grid = chain.grid(spacing=psize, data_names=[i],pixel_register=True)
                grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)
                tif_ = grids[i].rename(easting = 'x',northing='y')
                tif_.rio.to_raster('/home/ggrl/Desktop/grids/3022/100m_25k/'+i+'_'+row.id_folha+'.tif')

In [195]:
interpolar(df,g_3022)

SF23_Y_A_III_4 start
CTCOR
eU
eTH
MDT
KPERC


In [203]:
plt.figure(row.id_folha)
grids['MDT'].MDT.plot(cmap='terrain',vmin=0,vmax=800)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(576836.79, 601891.15, 7428463.420000001, 7456277.550000001)

In [ ]:
interpolar(df,gLine_1105,validate=True)